In [355]:
import os,sys,math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.discrete.discrete_model as lg
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score as auc
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import r2_score as r2
from sklearn.model_selection import train_test_split as split
%matplotlib inline
e=math.exp
ln=math.log

plt.style.use("fivethirtyeight")

In [502]:
l=pd.read_csv('data/loans_imputed.csv')

dums=pd.get_dummies(l['purpose'],drop_first=True)
#dums.iloc[0]=dums.iloc[0].apply(lambda x: 1)


l=pd.concat([l,dums],axis=1).drop('purpose',axis=1)

sub=split(l,test_size=0.70)
lTrain=sub[1]
lTest=sub[0]

xTrain=lTrain.drop('not.fully.paid',axis=1)
xTrain=sm.add_constant(xTrain)
yTrain=lTrain['not.fully.paid']

xTest=lTest.drop('not.fully.paid',axis=1)
xTest=sm.add_constant(xTest)
yTest=lTest['not.fully.paid']


model1=lg.Logit(yTrain,xTrain).fit()

model1.summary()


Optimization terminated successfully.
         Current function value: 0.412049
         Iterations 6


1.9140090091603235

In [537]:
l=pd.read_csv('data/loans_imputed.csv')

#mask for profit = -1 when not.fully.paid is True
l['profit']=l['int.rate'].apply(lambda x: e(x*3)-1)
mask=l['not.fully.paid']>0
l['profit'][mask]=l.profit[mask].apply(lambda x: -1)

sub=split(l,test_size=0.70)

lTrain=sub[1]
lTest=sub[0]

xTrain=lTrain['int.rate']
xTrain=sm.add_constant(xTrain)
yTrain=lTrain['not.fully.paid']

xTest=lTest['int.rate']
xTest=sm.add_constant(xTest)
yTest=lTest['not.fully.paid']

model2=lg.Logit(yTrain,xTrain).fit()

yPred=model2.predict(xTest)

yPred.max()

lTest['predicted.risk']=yPred

#test set area under curve of the bivariate model

auc(yTest,yPred)

highInterest=lTest[lTest['int.rate']>0.15]


temp=lTest.sort_values('predicted.risk').head(100)
temp['not.fully.paid'].sum()

C:\Users\lucasfermo\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Optimization terminated successfully.
         Current function value: 0.423052
         Iterations 6


C:\Users\lucasfermo\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


37